# Phase 2: Classifier

In [ ]:
!pip install git+https://github.com/marcoancona/DeepExplain

In [ ]:
!pip install kneed

In [ ]:
!pip install catboost


In [ ]:
#!pip uninstall h5py
!pip install h5py==2.10.0

In [ ]:
#!pip install git+https://github.com/albermax/innvestigate
from google.colab import drive
drive.mount('/content/drive/')
import os

os.chdir('/content/drive/My Drive/CNV')

In [ ]:
!ls

In [ ]:
"""
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorflow_version 1
#Start tensorboard
%tensorboard — logdir logs
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter('ignore')
"""

In [ ]:
seed=0
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
#from tensorflow import set_random_seed
#set_random_seed(seed)

In [ ]:
#!pip install --upgrade scikit-learn
#!pip install --upgrade imblearn

In [ ]:
#! pip install scikit-learn==0.21.1
!pip install --upgrade scikit-learn
! pip install -U imbalanced-learn

In [ ]:
%matplotlib inline  

import imp
import numpy as np
import os
import keras
import keras.backend
import keras.models
#import innvestigate
#import innvestigate.utils as iutils
#!pip install scikit-learn==0.20.0
import sklearn
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras import regularizers
from keras.models import Model,Sequential
from keras.layers import Input,Dense,Dropout
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.utils import plot_model
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import keras.models as mod
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import TensorBoard
from keras.utils.vis_utils import plot_model
#from sklearn.externals import joblib
import joblib
drpRate = 0.55

In [ ]:
#%tensorflow_version 1


In [ ]:
#Initially 0
np.random.seed(seed)
df = pd.read_csv('./dataset/TCGA_BRCA_CNV.gz',compression='gzip',sep='\t',index_col=0)
df = df.reindex(sorted(df.columns), axis=1)
df=df.T
print("CNV Matrix Before:", df.shape)
df2=pd.read_csv('./dataset/BRCA_clinicalMatrix.gz',compression='gzip',sep='\t',index_col=0)
print("Clinical Data Before:", df2.shape)
k=df2.columns.get_loc('PAM50Call_RNAseq')
df2=df2[df2.iloc[:,k].isna()==False]
#df2=df2[df2.iloc[:,k]!='Normal']
#d11=list(set(df.index)-set(df2.index)) #Comment this, if you want to test on unknown sample (generalize model)
commonIndexes=sorted(list(set(df2.index).intersection(set(df.index)))) 
df=df.loc[commonIndexes]
df2=df2.loc[commonIndexes] 
#df=df.drop(d)                         #Comment this, if you want to test on unknown sample (generalize model)
print("CNV After:", df.shape)
print("Clinical Data After:", df2.shape)
print("CNV and Clinical Data corresponds to same Patients: ", False if False in (df.index==df2.index) else True)
unique_elements, counts_elements = np.unique(df2.iloc[:,k], return_counts=True)
print("Labels", unique_elements, counts_elements)

In [ ]:
CNV45 = ['ZMAT4', 'ADRB3', 'FGF4', 'ZFPM2', 'LETM2', 'BRF2', 'C8orf86', 'RPS20P22', 'PROSC', 'RNA5SP58', 'ZNF703', 'MIR1208', 'FADD', 'IKZF3', 'SNX27', 'MYC', 'GRB7', 'ERBB2', 'PGAP3', 'RBM4', 'STAR', 'MIR4728', 'ANO1', 'MYEOV', 'STARD3', 'ASH2L', 'GPR124', 'ERLIN2', 'TCAP', 'CCND1', 'VPS45', 'EIF4EBP1', 'PPAPDC1B', 'TACC1', 'CDK12', 'RN7SL709P', 'NOTCH2', 'WHSC1L1', 'GOT1L1', 'DDHD2', 'MIEN1', 'PTK2', 'FGFR1', 'PPP1R1B', 'LSM1', 'PNMT']
sortedGenes=['ZFPM2', 'RBM4', 'MYC', 'VPS45', 'RNA5SP58', 'MIR1208', 'SNX27', 'NOTCH2', 'PTK2', 'STAR', 'EIF4EBP1', 'MYEOV', 'PROSC', 'CCND1', 'DDHD2', 'PPAPDC1B', 'RN7SL709P', 'TACC1', 'GOT1L1', 'ADRB3', 'LETM2', 'BRF2', 'GPR124', 'ASH2L', 'RPS20P22', 'FGF4', 'ANO1', 'C8orf86', 'FADD', 'FGFR1', 'LSM1', 'ZNF703', 'WHSC1L1', 'PROSC', 'TACC1', 'ZNF703', 'BRF2', 'LSM1', 'DDHD2', 'PPAPDC1B', 'RN7SL709P', 'EIF4EBP1', 'ERLIN2', 'GPR124', 'ASH2L', 'ANO1', 'C8orf86', 'FADD', 'MIEN1', 'CDK12', 'MIR4728', 'PGAP3', 'PNMT', 'PPP1R1B', 'TCAP', 'IKZF3', 'GRB7', 'STARD3', 'ERBB2']
from pandas import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
genes = df[sortedGenes]#df[list(set(genes_52))]
plt.figure(figsize=(10,7))
cm = genes.corr()
sns.heatmap(cm, cmap = 'magma_r')#, square=True)
plt.yticks(rotation=0)
plt.xticks(rotation=90)
#plt.savefig('27genes.png', dpi=600, bbox_inches='tight')

In [ ]:
#dfFinal1=df2[['PAM50Call_RNAseq', 'OS', 'OS.time']] #+  df[list(set(genes_52))]
SelectedWOSMOTE = ['CASC8', 'RN7SL563P', 'SNORA70|ENSG00000252526.1', 'ZNF704', 'TRHR', 'RPL30', 'RNA5SP76', 'RNA5S10', 'RUSC1', 'ANKRD46', 'MIR5194', 'LRRC52', 'CRCT1', 'S100A12', 'EIF3E', 'SNORA32|ENSG00000206776.1', 'MIR548A3', 'SNX16', 'GREM2', 'SUCO', 'NUCKS1', 'C8orf37', 'KHDRBS3', 'SPRTN', 'KIFAP3', 'JRK', 'KLHL38', 'MIR30D', 'OR2M3', 'LCE1B', 'OR2M5', 'OR2T11', 'SPRR2E', 'CSMD3', 'PCAT2', 'OR10K1', 'ZNF706', 'TARBP1', 'MIR1205', 'SLURP1', 'OR2B11', 'CHRAC1', 'MIR5681A', 'VPS45', 'PABPC1', 'TCEB1', 'MIR2053', 'TBCE', 'MTBP', 'HIST2H3C', 'AKT3', 'RNA5SP277', 'CGN', 'ATAD2', 'SCARNA4|ENSG00000252808.1', 'EIF3H', 'PRRC2C', 'RNF187', 'SOAT1', 'SNTB2', 'UBE2V2', 'ASPH', 'SLC26A9', 'LINC00964', 'NBPF8', 'TAF2', 'ZHX2']
dfFinal= df[SelectedWOSMOTE]
print(dfFinal.head())
#dfFinal1.to_csv('./KMPLOT1.xlsx')

In [ ]:
dfFinal2=df2[['PAM50Call_RNAseq', 'OS', 'OS.time']] #+  df[list(set(genes_52))]

print(dfFinal2.head())
#dfFinal2.to_csv('./KMPLOT2.xlsx')

In [ ]:
dfFinal.columns

In [ ]:
for i in dfFinal.columns:
  print(i)

In [ ]:
X=np.array(dfFinal.values)
scaler=StandardScaler()
XScaled=X#scaler.fit_transform(X)
print(XScaled.mean(axis = 0), XScaled.std(axis = 0))

In [ ]:
df[list(set(dfFinal.columns))].T.to_csv('X_44FINAL.csv', sep='\t')
df2.iloc[:,k].to_csv('Y_44FINAL.csv', header=True, index=True, sep='\t')

In [ ]:
#Encoding labels to numerical  values
SampleIDs = np.array(df2.index )
Y_temp=np.array(df2.values[:,k])
print("TARGET LABEL SIZE BEFORE: ", Y_temp.shape)
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(Y_temp)
print("TARGET LABEL SIZE BEFORE: ", Y.shape)
print("Labels before and after:", Y_temp[0:9], Y[0:9])
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(le_name_mapping)

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
SampleIDs = SampleIDs.reshape(len(Y), 1)
Y = Y.reshape(len(Y), 1)
Y_onehot = onehot_encoder.fit_transform(Y)
print(Y_onehot.shape, SampleIDs.shape)

In [ ]:
Y_onehot=Y

In [ ]:
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from collections import Counter
#seed=999999#27112017 acc= 68.5
# seed=0 acc =67.1
#seed=1, acc=69
seed=13 #0-69.8, 1-70.8, 2-69, 9-70.50, 26122021-71.48
import os
import numpy as np
import tensorflow as tf
import random as rn
from catboost import CatBoostClassifier

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
rn.seed(seed)
#tf.set_random_seed(seed)
tf.random.set_seed(seed)
n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [ ]:
for seed in range(13,14):
                    print("\n\n******************************************SEED", seed,"\n")
                    os.environ['PYTHONHASHSEED']=str(seed)
                    np.random.seed(seed)
                    rn.seed(seed)
                    #tf.set_random_seed(seed)
                    tf.random.set_seed(seed)

                    n_splits = 5

                    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
                    c=np.zeros([5,5])
                    cm=[]
                    i = 0
                    acc=[]
                    total=0
                    for train_idx, test_idx in skf.split(X, Y):
                      X_train = X[train_idx]
                      Y_train = Y_onehot[train_idx]
                      X_test = X[test_idx]
                      Y_test = Y_onehot[test_idx]


                      
                      unique_elements, counts = np.unique(Y_train, return_counts=True)
                      from imblearn.over_sampling import ADASYN
                      from imblearn.combine import SMOTEENN
                      from collections import Counter
                      counts = Counter(Y_train.reshape(Y_train.shape[0]))
                      print(counts)
                      sm = SMOTE(random_state=seed)#, sampling_strategy= {0:int(counts[0]*1), 1:int(counts[1]*1), 2:counts[2], 3:int(counts[3]*1), 4:int(counts[4]*2.5)})


                      X_train_res,Y_train_res=X_train,Y_train#sm.fit_resample(X_train,Y_train)

                      from lightgbm import LGBMClassifier
                      from sklearn import metrics
                      from sklearn.metrics import accuracy_score, classification_report
                      
                      
                      from catboost import CatBoostClassifier

                      model = CatBoostClassifier(verbose=0)#random_state=seed,   iterations=1000,          task_type="GPU", devices='0',verbose=0)
                      #####model = LGBMClassifier(random_state=seed)
                      model.fit(X_train_res, Y_train_res)
                      y_pred_cat = model.predict(X_test)
                      
                      print("Accuracy: ", metrics.accuracy_score(Y_test, y_pred_cat))
                      Accuracy5 =  "CatBoost "+str(metrics.accuracy_score(Y_test, y_pred_cat))
                      c1=confusion_matrix(Y_test, y_pred_cat)
                      print(confusion_matrix(Y_test, y_pred_cat))
                      total += metrics.accuracy_score(Y_test, y_pred_cat)
                      acc.append(metrics.accuracy_score(Y_test, y_pred_cat))
                      i += 1
                      cm.append( np.asarray(c1))
                      c += np.asarray(c1)
                    print('Average Accuracy: ', total/n_splits) 

                    # acc confidence interval for 52 genes
                    import numpy as np
                    acc= np.asarray(acc)
                    print("acc : %0.3f (+/- %0.3f)" % (acc.mean(), acc.std() * 1.96))

                    

In [ ]:
precision1 = []
precision2 = []
precision3 =[]
precision4=[]
precision5 =[]
recall1 =[]
recall2= []
recall3=[]
recall4=[]
recall5=[]
f1score1 =[]
f1score2 =[]
f1score3 =[]
f1score4 =[]
f1score5 =[]

for i in range(1,6):
   conf = cm[i-1]
   for j in range(1,6):
        p=eval("precision"+str(j))
        r=eval("recall"+str(j))
        f=eval("f1score"+str(j))
        p1=conf[j-1][j-1]/sum(conf[j-1][k] for k in range(0,5) )
        r1=conf[j-1][j-1]/sum(conf[k][j-1] for k in range(0,5))
        #print(p1,r1)
        p.append(p1)
        r.append(r1)
        f.append((2*p1*r1)/(p1+r1))
        

import seaborn as sns
sns.set()#style="whitegrid")
import pandas as pd
result = [                
                       ['Precision',precision1], 
                       ['Precision',precision2],
                       ['Precision',precision3],
                       ['Precision',precision4],
                       ['Precision',precision5],

                       ['Recall',recall1], 
                       ['Recall',recall2],
                       ['Recall',recall3],
                       ['Recall',recall4],
                       ['Recall',recall5],

                       ['F1-Score',f1score1], 
                       ['F1-Score',f1score2],
                       ['F1-Score',f1score3],
                       ['F1-Score',f1score4],
                       ['F1-Score',f1score5]              ]
#result = np.asarray(result)
#print(result, result.size)



xLabels = ['Basal']*5 + ['Her2']*5 + ['LumA']*5 +['lumB']*5 +['Normal']*5
xLabels  = ['All Classes']*5+xLabels * 3

finalResult  = pd.DataFrame({ 'Metric': ['Accuracy']*5+[i[0] for i in result for j in range(5)], 
                             'value':list(acc)+[j for i in result for j in i[1]], 'Labels': xLabels })
           
my_pal = {"Accuracy": "turquoise", "Precision": "cornflowerblue", "Recall":"violet", "F1-Score": "coral"}


ax = sns.boxplot(x= "Labels", y = "value", hue = "Metric"  , data = finalResult, linewidth=2, orient="v", palette=my_pal)

plt.ylabel('Metrics Score')
plt.xlabel('Breast Cancer Subtype')
plt.ylim(0.0,1.025)
ax.xaxis.grid(True)
plt.savefig('figures/TCGAboxPlotMethylRF.png', dpi = 600)


In [ ]:
from matplotlib import pyplot as plt
!pip install mlxtend --upgrade --no-deps
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
plt.figure()


fig, ax = plot_confusion_matrix(conf_mat=c.astype(int),
                                colorbar=True,
                                show_absolute=True,
                                show_normed=True, class_names = ['Basal', 'Her2', 'LumA', 'LumB', 'Normal'])

#plt.show()
plt.savefig('figures/TCGACM_methyRF.png', bbox_inches = 'tight', dpi = 600)

In [ ]:
c=c.astype(int)
total=c.sum(axis=1).astype(int)
total[4]

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report
y_true=np.asarray([0]*(total[0])+[1] *(total[1])+[2]*(total[2])+[3]*(total[3])+ [4]*(total[4]))
y_pred = np.asarray([0]*c[0,0]+[1]*c[0,1]+[2]*c[0,2]+[3]*c[0,3]+[4]*c[0,4]+
                    [0]*c[1,0]+[1]*c[1,1]+[2]*c[1,2]+[3]*c[1,3]+[4]*c[1,4]+
                    [0]*c[2,0]+[1]*c[2,1]+[2]*c[2,2]+[3]*c[2,3]+[4]*c[2,4]+
                    [0]*c[3,0]+[1]*c[3,1]+[2]*c[3,2]+[3]*c[3,3]+[4]*c[3,4]+
                   [0]*c[4,0]+[1]*c[4,1]+[2]*c[4,2]+[3]*c[4,3]+[4]*c[4,4])
print(classification_report(y_true, y_pred, target_names=['Basal', 'Her2', 'LumA', 'LumB', 'Normal'],digits=5,output_dict=True))
clf_report=classification_report(y_true, y_pred, target_names=['Basal', 'Her2', 'LumA', 'LumB', 'Normal'],digits=5,output_dict=True)


import pandas as pd

fig = plt.figure(figsize=(6,4))
sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True, fmt='.3g', center = True)
plt.savefig('figures/TCGAHeatmapRF.png',dpi=600, bbox_inches = "tight")

plt.show()

#SHAP

In [ ]:
 !pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
total = 0
i = 0
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
if True:
  X_train = X
  Y_train = Y_onehot

  from imblearn.over_sampling import ADASYN
  sm = SMOTE(random_state=seed)
  X_train_res,Y_train_res=sm.fit_resample(X_train,Y_train)

  from catboost import CatBoostClassifier
  model = CatBoostClassifier(random_state=seed, iterations=2000, task_type="GPU", devices='0',verbose=0)
  
  
  model.fit(X_train_res, Y_train_res) 
  #model.fit(X_train_res, Y_train_res)
  print("done")
  #X_test = np.reshape(X_test.shape[0])
  y_pred_cat = model.predict(X_train)
  print("Accuracy: ", metrics.accuracy_score(Y_train, y_pred_cat))
  Accuracy5 =  "CatBoost "+str(metrics.accuracy_score(Y_train, y_pred_cat))
  print(confusion_matrix(Y_train, y_pred_cat))
  total += metrics.accuracy_score(Y_train, y_pred_cat)
  i += 1

print('Average Accuracy: ', total/1)

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
import numpy as np
np.random.seed(seed)

import sklearn
import shap
from sklearn.model_selection import train_test_split

# print the JS visualization code to the notebook
shap.initjs()

#explainer = shap.TreeExplainer(model, X)

# use Kernel SHAP to explain test set predictions
#explainer = shap.KernelExplainer(classifier.predict, X_train_res, random_state=seed)
#shap_values = explainer.shap_values(X_test, nsamples=100)



In [ ]:
explainer = shap.KernelExplainer(model.predict_proba, X, random_state=seed) #clf.predict_proba 

In [ ]:
#explainer = shap.TreeExplainer(model, X)

In [ ]:
seed

In [ ]:
import numpy as np  
np.random.seed(seed)   
shap_values = explainer.shap_values(X, nsamples=450, random_state=seed)   
#explainer.shap_values(Pool(X, y, cat_features=cat_features))   

In [ ]:
#from sklearn.externals import joblib
#shap_values = ex.shap_values(X_test.iloc[0, :])
import pickle
filename = 'shapFinal52' 
outfile = open(filename,'wb')
pickle.dump(explainer,outfile) 
pickle.dump(shap_values,outfile)
outfile.close()

In [ ]:
import shap
#from sklearn.externals import joblib
#shap_values = ex.shap_values(X_test.iloc[0, :])
import pickle
filename = 'shapFinal52'
outfile = open(filename,'rb')
explainer = pickle.load(outfile)
shap_values = pickle.load(outfile)
outfile.close()

In [ ]:
X_test=X
Y_test=Y_onehot

In [ ]:
len(shap_values[0][0])

In [ ]:
#shap.initjs()
# plot the SHAP values for the Setosa output of the first instance
#shap.force_plot(explainer.expected_value[0], shap_values[0][3,:], X_test.iloc[3,:])

In [ ]:
shap.summary_plot(shap_values, X_test, feature_names = dfFinal.columns, max_display=250, class_names=['Basal', 'Her2', 'LuminalA', 'LuminalB', 'Normal'], show=False)
plt.savefig('shap_Final52.png', dpi=600, bbox_inches='tight')
plt.show()
#['S100Z', 'MSLNL', 'BCL10', 'WDR92', 'ACBD4', 'KLHDC4', 'SCARNA22', 'AP4S1', 'KIAA2026', 'L3HYPDH', 'SNORA19|ENSG00000251974.1', 'NDRG3', 'CKMT1A', 'KATNB1', 'MRPL21','RPL8', 'AFMID', 'RPS20P22', 'AHI1', 'RN7SKP89'
#, 'MIR1208', 'GAL', 'SLC2A14', 'SNORA76|ENSG00000266402.2', 'FAM92A1', 'CACNG5', 'SGSH', 'STARD3','DHCR7', 'ZHX1']
#'SNORA8|ENSG00000207410.1', 'MIR3141', 'SH3YL1', 'ALOX12B', 'ARL5C', 'TUBA1A', 'RPS6KC1'

In [ ]:
X_test0= np.asarray([X_test[i] for i in range(X_test.shape[0]) if Y_test[i] == 0 ])
index0= [i for i in range(X_test.shape[0]) if Y_test[i] == 0 ]
X_test1= np.asarray([X_test[i] for i in range(X_test.shape[0]) if Y_test[i] == 1 ])
index1= [i for i in range(X_test.shape[0]) if Y_test[i] == 1 ]
X_test2= np.asarray([X_test[i] for i in range(X_test.shape[0]) if Y_test[i] == 2 ])
index2= [i for i in range(X_test.shape[0]) if Y_test[i] == 2 ]
X_test3= np.asarray([X_test[i] for i in range(X_test.shape[0]) if Y_test[i] == 3 ])
index3= [i for i in range(X_test.shape[0]) if Y_test[i] == 3 ]
X_test4= np.asarray([X_test[i] for i in range(X_test.shape[0]) if Y_test[i] == 4 ])
index4= [i for i in range(X_test.shape[0]) if Y_test[i] == 4 ]



In [ ]:
X_test0.shape, np.array(index0).shape

In [ ]:
genes_52=dfFinal.columns

In [ ]:
shap.summary_plot(shap_values[0][index0], np.asarray(X_test0), feature_names = genes_52, max_display=10, show=False, color_bar=False)
plt.title("BASAL", loc='right')
plt.savefig('shap52_basal_onlyBasalPatients.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
shap.summary_plot(shap_values[0], X_test, feature_names = genes_52, max_display=20, show=False, color_bar=False)
plt.title("BASAL", loc='right')
plt.savefig('shap52_basal_allPatients.png', dpi=600, bbox_inches='tight')
plt.show()
#['S100Z', 'MSLNL', 'BCL10', 'WDR92', 'ACBD4', 'KLHDC4', 'SCARNA22', 'AP4S1', 'KIAA2026', 'L3HYPDH', 'SNORA19|ENSG00000251974.1', 'NDRG3', 'CKMT1A', 'KATNB1', 'MRPL21','RPL8', 'AFMID', 'RPS20P22', 'AHI1', 'RN7SKP89']#, 'MIR1208', 'GAL', 'SLC2A14', '']

In [ ]:
shap.summary_plot(shap_values[1][index1], np.asarray(X_test1), feature_names = genes_52, max_display=10, show=False, color_bar=False)
plt.title("HER2", loc='right')
plt.savefig('shap52_her2_onlyHer2Patients.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
shap.summary_plot(shap_values[1], X_test, feature_names = genes_52, max_display=30, show=False, color_bar=False)
plt.title("HER2", loc='right')
plt.savefig('shap52_her2_allPatients.png', dpi=600, bbox_inches='tight')
plt.show()
#['S100Z', 'MSLNL', 'BCL10', 'WDR92', 'ACBD4', 'KLHDC4', 'SCARNA22', 'AP4S1', 'KIAA2026', 'L3HYPDH', 'SNORA19|ENSG00000251974.1', 'NDRG3', 'CKMT1A', 'KATNB1', 'MRPL21','RPL8', 'AFMID', 'RPS20P22', 'AHI1', 'RN7SKP89']#, 'MIR1208', 'GAL', 'SLC2A14', '']

In [ ]:
shap.summary_plot(shap_values[2][index2], np.asarray(X_test2), feature_names = genes_52, max_display=20, show=False, color_bar=False)
plt.title("LUMA", loc='right')
plt.savefig('shap52_lumA_onlyLumAPatients.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
shap.summary_plot(shap_values[2], X_test, feature_names = genes_52, max_display=20, show=False, color_bar=False)
plt.title("LUMA", loc='right')
plt.savefig('shap52_lumA_allPatients.png', dpi=600, bbox_inches='tight')
plt.show()
#['S100Z', 'MSLNL', 'BCL10', 'WDR92', 'ACBD4', 'KLHDC4', 'SCARNA22', 'AP4S1', 'KIAA2026', 'L3HYPDH', 'SNORA19|ENSG00000251974.1', 'NDRG3', 'CKMT1A', 'KATNB1', 'MRPL21','RPL8', 'AFMID', 'RPS20P22', 'AHI1', 'RN7SKP89']#, 'MIR1208', 'GAL', 'SLC2A14', '']


In [ ]:
#CORRECT
shap.summary_plot(shap_values[3][index3], np.asarray(X_test3), feature_names = genes_52, max_display=20, show=False, color_bar=False)
plt.title("LUMB", loc='right')
plt.savefig('shap52_lumB_onlyLumBPatients.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
#CORRECT
shap.summary_plot(shap_values[3], X_test, feature_names = genes_52, max_display=20, show=False, color_bar=False)
plt.title("LUMB", loc='right')
plt.savefig('shap52_lumB_allPatients.png', dpi=600, bbox_inches='tight')
plt.show()
#['S100Z', 'MSLNL', 'BCL10', 'WDR92', 'ACBD4', 'KLHDC4', 'SCARNA22', 'AP4S1', 'KIAA2026', 'L3HYPDH', 'SNORA19|ENSG00000251974.1', 'NDRG3', 'CKMT1A', 'KATNB1', 'MRPL21','RPL8', 'AFMID', 'RPS20P22', 'AHI1', 'RN7SKP89']#, 'MIR1208', 'GAL', 'SLC2A14', '']

In [ ]:
shap.summary_plot(shap_values[4][index4], np.asarray(X_test4), feature_names = genes_52, max_display=20, show=False)
plt.title("NORMAL", loc='right')
plt.savefig('shap52_normal_onlyNormalPatients.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
shap.summary_plot(shap_values[4], X_test, feature_names = genes_52, max_display=20, show=False)
plt.title("NORMAL", loc='right')
plt.savefig('shap52_normal_allPatients.png', dpi=600, bbox_inches='tight')
plt.show()